# Potencial Ornitológico Fueguino
### **Autor:** Pablo Jusim

# Script de preprocesamiento de observaciones de iNaturalist
Se realizan virificaciones antes de obtener el dataframe final.

La salida de este script será un dataframe de pandas donde cada fila representará una observación de iNaturalist.

El dataframe final tendrá las siguientes **columnas**: id_obs, id_celda, especie, familia, orden, fecha

## Importaciones

In [42]:
import pandas as pd
import sys
from pathlib import Path

# Modulos propios
sys.path.append(str(Path('..')/'src'))
import asociar_grilla
import utils

## Carga de datos

In [43]:
archivo = '../data/raw/obs_iNat.csv'
df_base = pd.read_csv(archivo)
df_base.head()

,id,uuid,observed_on_string,observed_on,time_observed_at,user_id,user_name,created_at,quality_grade,captive_cultivated,...,place_admin2_name,species_guess,scientific_name,common_name,iconic_taxon_name,taxon_id,taxon_order_name,taxon_family_name,taxon_genus_name,taxon_subspecies_name
0,43929,bf9ff5ac-0a0f-45eb-9c7b-e86e9298d192,"January 06, 2011 16:29",2011-01-06,2011-01-06 19:29:00 UTC,4083,Liam Quinn,2011-12-18 19:58:12 UTC,research,False,...,Ushuaia,Rufous-collared Sparrow,Zonotrichia capensis,Chingolo,Aves,9183,Passeriformes,Passerellidae,Zonotrichia,NaN
1,44127,dfa80126-6707-43dd-80e8-cf6230a6b41a,"January 06, 2011 12:44",2011-01-06,2011-01-06 15:44:00 UTC,4083,Liam Quinn,2011-12-19 03:36:06 UTC,research,False,...,Ushuaia,Austral Thrush,Turdus falcklandii,Zorzal patagónico,Aves,12751,Passeriformes,Turdidae,Turdus,NaN
2,143793,2c967eea-825c-4707-9c9a-a0aa157cfe10,2011-12-31 12:27:52,2011-12-31,2011-12-31 20:27:52 UTC,10201,edwardrooks,2012-11-04 22:32:17 UTC,research,False,...,Ushuaia,Black-necked Swan,Cygnus melancoryphus,Cisne cuello negro,Aves,72789,Anseriformes,Anatidae,Cygnus,NaN
3,143794,818697e7-2cff-4004-af67-b70b3ec39baf,2011-12-30 15:12:17,2011-12-30,2011-12-30 23:12:17 UTC,10201,edwardrooks,2012-11-04 22:32:18 UTC,research,False,...,Rió Grande,Southern Lapwing,Vanellus chilensis,Tero,Aves,4867,Charadriiformes,Charadriidae,Vanellus,NaN
4,335952,7efa102f-eb4a-45e5-a17b-7bb68db01940,2009-11-19,2009-11-19,NaN,15211,Carmelo López Abad,2013-07-18 00:40:23 UTC,research,False,...,Ushuaia,Crested Duck,Lophonetta specularioides,Pato Crestón,Aves,72994,Anseriformes,Anatidae,Lophonetta,NaN


## Limpieza de datos

### Revisión y eliminación de columnas no útiles

#### Revisión del contenido de columnas con posibles errores y eliminación de las mismas

##### Verificaciones

In [44]:
# Columna "quality_grade": segun la busqueda realizada en iNat, los datos deberian estar todos en "research"
print(f'quality_grade: {df_filtrada_1['quality_grade'].unique()}')

# Columna "captive_cultivated": ninguna observacion deberia ser "true" (aves cautivas)
print(f'captive: {df_filtrada_1['captive_cultivated'].unique()}')

# Columna "private_place_guess": ninguna observacion deberia tener datos ya que la búsqueda solo incluyó ubicaciones públicas
print(f'private_place_guess: {df_filtrada_1["private_place_guess"].unique()}')

# Columnas "private_latitude" y "private:longitude": ninguna observacion deberia tener datos ya que la búsqueda solo incluyó ubicaciones públicas
print(f'private_latitude: {df_filtrada_1["private_latitude"].unique()}')
print(f'private_longitude: {df_filtrada_1["private_longitude"].unique()}')

# Columna "geoprivacy": ninguna observacion deberia tener datos ya que la búsqueda solo incluyó ubicaciones públicas
print(f'geoprivacy: {df_filtrada_1["geoprivacy"].unique()}')

# Columna "taxon_geoprivacy": las observaciones deberian ser "open" o carecer el dato
print(f'taxon_geoprivacy: {df_filtrada_1["taxon_geoprivacy"].unique()}')

# Columna "coordinates_obscured": las observaciones deberian ser "false" o carecer el dato
print(f'coordinates_obscured: {df_filtrada_1["coordinates_obscured"].unique()}')

# Columna "place_town_name": se revisa si hay datos en la columna, sino se elimina
print(f'place_town_name: {df_filtrada_1["place_town_name"].unique()}')

# Columna "place_county_name": los valores deberian ser los departamentos de TIerras del Fuego
print(f'place_county_name: {df_filtrada_1["place_county_name"].unique()}')

# Columna "place_state_name": los valores deberian ser "Tierra del Fuego"
print(f'place_state_name: {df_filtrada_1["place_state_name"].unique()}')

# Columna "iconic_taxon_name": los valores deberian ser "Aves"
print(f'iconic_taxon_name: {df_filtrada_1["iconic_taxon_name"].unique()}')



quality_grade: ['research']
captive: [False]
private_place_guess: [nan]
private_latitude: [nan]
private_longitude: [nan]
geoprivacy: [nan]
taxon_geoprivacy: ['open' nan]
coordinates_obscured: [False]
place_town_name: [nan]
place_county_name: ['Ushuaia' 'Rió Grande']
place_state_name: ['Tierra del Fuego']
iconic_taxon_name: ['Aves']


In [45]:
print(f'Filas totales: {len(df_filtrada_1)}')
print(f'Columnas totales: {len(df_filtrada_1.columns)}')

Filas totales: 15651
Columnas totales: 26


### Revisión de valores de filas

##### Precisión

In [46]:
# Precision de las observaciones: Se descartan observaciones con una precisión menor
# a la mitad del tramaño de la celda (2500 m o más)
df_filtrada_2 = df_filtrada_1[df_filtrada_1['positional_accuracy'] < 2500]

##### Nombres científicos
Los nombres cientificos constan de dos palabras, el genero y el epíteto específico.
Si hay una tercera palabra, esta puede corresponder a una subespecie.
Tener observaciones de la misma especie con dos o tres palabras puede complicar el conteo

In [47]:
# Cambiar el nombre de la columna "scientific_name" a "scientific_name_sub"
df_filtrada_3 = df_filtrada_2.rename(columns={'scientific_name': 'scientific_name_sub'})

# Agregar la columna "scientific_name" con el nombre científico de la especie (solo las dos primeras palabras)
df_filtrada_3['scientific_name'] = df_filtrada_3[
    'scientific_name_sub'].apply(
        lambda x: ' '.join(x.split()[:2]) if isinstance(x, str) else x)

df_filtrada_3.head()

,id,observed_on,user_id,quality_grade,captive_cultivated,place_guess,latitude,longitude,positional_accuracy,private_place_guess,...,place_state_name,scientific_name_sub,common_name,iconic_taxon_name,taxon_id,taxon_order_name,taxon_family_name,taxon_genus_name,taxon_subspecies_name,scientific_name
6,336399,2009-11-21,15211,research,False,"Beagle Channel, Ushuaia, Tierra del Fuego prov...",-54.867678,-67.466637,38.0,NaN,...,Tierra del Fuego,Leucocarbo magellanicus,Cormorán cuello negro,Aves,1289600,Suliformes,Phalacrocoracidae,Leucocarbo,NaN,Leucocarbo magellanicus
22,568157,2013-12-22,10201,research,False,"Tierra del Fuego National Park, Ushuaia",-54.828777,-68.558121,1225.0,NaN,...,Tierra del Fuego,Enicognathus ferrugineus,Cachaña,Aves,19262,Psittaciformes,Psittacidae,Enicognathus,NaN,Enicognathus ferrugineus
23,568760,2013-12-22,10201,research,False,"Tierra del Fuego National Park, Ushuaia",-54.853885,-68.576660,611.0,NaN,...,Tierra del Fuego,Spinus barbatus,Cabecitanegra austral,Aves,145321,Passeriformes,Fringillidae,Spinus,NaN,Spinus barbatus
24,568802,2013-12-22,10201,research,False,"Tierra del Fuego National Park, Ushuaia",-54.844298,-68.564987,305.0,NaN,...,Tierra del Fuego,Lessonia rufa,Sobrepuesto austral,Aves,17310,Passeriformes,Tyrannidae,Lessonia,NaN,Lessonia rufa
26,568816,2013-12-22,10201,research,False,"Tierra del Fuego National Park, Ushuaia",-54.854478,-68.573914,2446.0,NaN,...,Tierra del Fuego,Zonotrichia capensis,Chingolo,Aves,9183,Passeriformes,Passerellidae,Zonotrichia,NaN,Zonotrichia capensis


##### Eliminación de columnas inútiles o sin paralelo en la otra fuente de datos

In [48]:
df_filtrada_4 = df_filtrada_3[['common_name', 'scientific_name', 'latitude', 'longitude', 'observed_on']]

# Ver resultado parcial
df_filtrada_4.head()

,common_name,scientific_name,latitude,longitude,observed_on
6,Cormorán cuello negro,Leucocarbo magellanicus,-54.867678,-67.466637,2009-11-21
22,Cachaña,Enicognathus ferrugineus,-54.828777,-68.558121,2013-12-22
23,Cabecitanegra austral,Spinus barbatus,-54.853885,-68.576660,2013-12-22
24,Sobrepuesto austral,Lessonia rufa,-54.844298,-68.564987,2013-12-22
26,Chingolo,Zonotrichia capensis,-54.854478,-68.573914,2013-12-22


##### Ver resumen de valores

In [49]:
# Ver la cantidad de datos faltantes en cada fila
print(df_filtrada_4.isnull().sum())

df_filtrada_4.describe()

common_name        88
scientific_name     0
latitude            0
longitude           0
observed_on         0
dtype: int64


,latitude,longitude
count,8087.000000,8087.000000
mean,-54.695671,-68.166464
std,0.356735,0.473518
min,-55.102000,-68.605735
25%,-54.838421,-68.363570
50%,-54.814984,-68.313788
75%,-54.796034,-68.179083
max,-52.659491,-63.811422


No se observan datos faltantes en columnas de importancia para el análisis. No se observan valores de columnas numéricas fuera del rango esperado

## Asociar la celda de la grilla a cada observación

In [50]:
# Punto de partida: directorio actual (notebooks/)
BASE_DIR = Path.cwd()
# Subir un nivel al directorio raíz del proyecto
PROYECTO_DIR = BASE_DIR.parent
# Construir ruta completa al archivo GPKG
DIR_DATOS = PROYECTO_DIR / 'data' / 'interim'

nombre_grilla = 'grilla_tdf_vacia.gpkg'
ruta_grilla = DIR_DATOS / nombre_grilla

In [51]:
# Asociar grilla a los datos de observaciones

df_grillada =asociar_grilla.assign_grid_cell_ids(
    grilla = ruta_grilla,
    datos_georef = df_filtrada_4,
    grid_id_field = 'grid_id'
)

## Exportar el data frame resultante

In [52]:
# Exportar el DataFrame df_filtrada_3
df_grillada.to_csv('../data/interim/data_inat.csv', index=False)